In [1]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import Normalizer, MinMaxScaler,StandardScaler
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
import xgboost as xgb

Using TensorFlow backend.


In [2]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\DC_ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name
day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)

y=df['Sales']

In [3]:
df.head()
df.shape

(942, 16)

In [4]:
X=df.drop(['id','DayOfWeek','Customers','Sales'],inplace=False,axis=1)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,10,2013,1,0,0,0,0,0,1,0
1,10,0,10,2013,1,0,0,0,0,0,0,1
2,10,0,8,2013,1,0,0,0,0,1,0,0
3,10,0,8,2013,1,1,0,0,0,0,0,0
4,10,0,1,2013,1,0,0,1,0,0,0,0


In [5]:
for obs in range(1,8):
    X["Sales_T"+str(obs)]=df['Sales'].shift(obs)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7
0,0,0,10,2013,1,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,0,10,2013,1,0,0,0,0,0,0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,10,0,8,2013,1,0,0,0,0,1,0,0,60436.0,0.0,NaN,NaN,NaN,NaN,NaN
3,10,0,8,2013,1,1,0,0,0,0,0,0,52848.0,60436.0,0.0,NaN,NaN,NaN,NaN
4,10,0,1,2013,1,0,0,1,0,0,0,0,55606.0,52848.0,60436.0,0.0,NaN,NaN,NaN


In [6]:
X['Mov_avg']=pd.rolling_mean(df['Sales'], window=7).shift(1) 
y=y[7:]
X=X[7:]
y.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)
X.head()
X.shape


(935, 20)

In [7]:
train_size=800
X_train, X_test, y_train, y_test =  X[0:train_size], X[train_size:len(X)], y[0:train_size], y[train_size:len(X)]
X_train.head()

std = preprocessing.StandardScaler().fit(X_train)
X_train=std.transform(X_train)
X_test=std.transform(X_test)

In [8]:
def RF_model(X_train, X_test, y_train, y_test):
    tree=RandomForestRegressor( random_state=2,criterion='mae')
    tree.fit(X_train,y_train)
    pred=tree.predict(X_test)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return pred

def NN_model(X_train, X_test, y_train, y_test):
    model = Sequential()
    model.add(Dense(30, input_dim=X_train.shape[1], activation='relu'))
    #model.add(Dense(40, activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(40, activation='linear'))
    #model.add(Dropout(0.1))
    model.add(Dense(10, activation='relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mae', optimizer='adam', metrics=['mae','mse'])
    model.fit(X_train, y_train, epochs=300, batch_size=20, verbose=0)
    prediction=model.predict(X_test, verbose=0)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'NN'})
    return pred

def GB_model(X_train, X_test, y_train, y_test):
    params = {'n_estimators': 500, 'max_depth': 40, 'min_samples_split': 20,
          'learning_rate': 0.01, 'loss': 'ls'}
    model = ensemble.GradientBoostingRegressor(**params)
    model.fit(X_train, y_train)
    prediction=model.predict(X_test)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return pred

def XG_model(X_train, X_test, y_train, y_test):
    model=xgb.XGBRegressor(  
        #learning_rate =0.1, n_estimators=500, max_depth=5, min_child_weight=1,gamma=0, 
        #subsample=0.8,colsample_bytree=0.8,nthread=4,scale_pos_weight=1,seed=27
    )
    model.fit(X_train, y_train)
    prediction=model.predict(X_test)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return pred

def MAE_score(y_true,pred1, pred2,pred3,pred4):
    mae1 = mean_absolute_error(y_true, pred1)
    mae2 = mean_absolute_error(y_true, pred2)
    mae3 = mean_absolute_error(y_true, pred3)
    mae4 = mean_absolute_error(y_true, pred4)
    return  [mae1.round(),mae2.round(),mae3.round(),mae4.round()] 

In [9]:
RF_pred=RF_model(X_train, X_test, y_train, y_test);
NN_pred=NN_model(X_train, X_test, y_train, y_test);
GB_pred=GB_model(X_train, X_test, y_train, y_test);
XG_pred=XG_model(X_train, X_test, y_train, y_test);

mae=MAE_score(y_test,RF_pred,NN_pred,GB_pred,XG_pred)
mae=pd.DataFrame(np.array(mae).reshape(1,4), columns = ["RF", "NN", "GB","XG"])
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae

,RF,NN,GB,XG,Best
0,3343.0,3771.0,3097.0,3570.0,GB


In [10]:
#Using aggregated SKU sales data, the best performaning model is selected based on Train/Test. 
#Then best model is used to predict full data set (but still only trained on X_train) 

X_norm=np.concatenate((X_train,X_test), axis=0)
bm_dict={}
for sku in range(0,1):        
        if  mae["Best"][0]=="RF":
            bm_dict[sku]=RF_model(X_train, X_norm, y_train, y);
                    
        elif mae["Best"][0]=="NN":
            bm_dict[sku]=NN_model(X_train, X_norm, y_train, y);
                                
        elif mae["Best"][0]=="GB":
            bm_dict[sku]=GB_model(X_train, X_norm, y_train, y); 
                        
        elif mae["Best"][0]=="XG":
            bm_dict[sku]=XG_model(X_train, X_norm, y_train, y);

bm_dict1=bm_dict[0][800:]
bm_dict1.reset_index(drop=True, inplace=True)

In [11]:
# Ceate Phase 2 data set which is comprised of both aggregate SKU data and forecasts (no epirical sales data)
dc_pred=pd.DataFrame(bm_dict[0])
sku_pred=pd.read_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred=sku_pred.iloc[:,1:]
X2=pd.concat([sku_pred,dc_pred,X],axis=1)

In [12]:
train_size=800
X2_train, X2_test, y2_train, y2_test =  X2[0:train_size], X2[train_size:len(X)], y[0:train_size], y[train_size:len(X)]

std2 = preprocessing.StandardScaler().fit(X2_train)
X2_train=std2.transform(X2_train)
X2_test=std2.transform(X2_test)

In [13]:
RF2_pred=RF_model(X2_train, X2_test, y2_train, y2_test);
NN2_pred=NN_model(X2_train, X2_test, y2_train, y2_test);
GB2_pred=GB_model(X2_train, X2_test, y2_train, y2_test);
XG2_pred=XG_model(X2_train, X2_test, y2_train, y2_test);

In [20]:
#Using forecasted data and no empirical sales data, the best performaning model is selected based on Train/Test. 

mae2=MAE_score(y2_test,RF2_pred,NN2_pred,GB2_pred,XG2_pred)
mae2=pd.DataFrame(np.array(mae2).reshape(1,4), columns = ["RF", "NN", "GB","XG"])
best2=mae2.idxmin(axis=1)
best2.astype('category')
mae2["Best"]=best2
both_mae=pd.concat([mae,mae2],axis=0)
both_mae.reset_index(drop=True, inplace=True)

# Phase 1 & 2 DC Models compared based on Test set MAE scores. 
both_mae['min']=both_mae.min(axis=1)
both_mae['improvement']= ((both_mae['min'][0]-both_mae['min'][1])/both_mae['min'][0] ).round(3)
both_mae

,RF,NN,GB,XG,Best,min,improvement
0,3343.0,3771.0,3097.0,3570.0,GB,3097.0,0.097
1,3004.0,2798.0,3116.0,2987.0,NN,2798.0,0.097


In [15]:
bm2_dict={}
for sku in range(0,1):
        
        if  mae2["Best"][0]=="RF":
            bm2_dict[sku]=RF2_pred            
        
        elif mae2["Best"][0]=="NN":
            bm2_dict[sku]=NN2_pred                   
            
        elif mae2["Best"][0]=="GB":
            bm2_dict[sku]=GB2_pred  

        elif mae["Best"][0]=="XG":
            bm2_dict[sku]=XG2_pred      

In [16]:
test_sales=y_test.loc[800:,]
test_sales.reset_index(drop=True, inplace=True)

In [17]:
tier_pred=pd.concat([bm_dict1,bm2_dict[0],test_sales],axis=1)
tier_pred.columns = ['DC_Phase1', 'DC_Phase2',"Sales"]
tier_pred.to_csv('C:\\Users\\lengada1\\NCSU\\DC_tier_predictions.csv')
tier_pred.round()

#Note record 874:  Total stores open = 7 with threw off both models since there were not training samples at this value
#1	4	6/4/2015	51712	5799	7	10	0


,DC_Phase1,DC_Phase2,Sales
0,65332.0,65709.0,65476
1,65753.0,66181.0,64438
2,46583.0,46952.0,44449
3,342.0,-0.0,0
4,52936.0,53112.0,55432
5,51522.0,51639.0,53261
6,52202.0,52713.0,49636
7,50429.0,50474.0,52817
8,55078.0,55601.0,53679
9,47324.0,47152.0,50023


In [18]:
# END NOTEBOOK

In [19]:
# The rest of this notebook is just a reference and aid to allow collaborators to save models and retrive and apply them without retraining. 
# The file names are not correct, thus just a reference. 

# also look at joblib.  https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
model_json = model.to_json()
with open("NCSU_DC.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("NCSU_DC.h5")
print("Saved model to disk")

NameError: name 'model' is not defined

In [ ]:
# load json and create model
json_file = open('NCSU_DC.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("NCSU_DC.h5")
print("Loaded model from disk")

In [ ]:
# evaluate loaded model on test data
loaded_model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
score = loaded_model.evaluate(Xs, y, verbose=0)
print("%s: %.0f"%  (loaded_model.metrics_names[1], score[1]))
print("%s: %.1f"%  (loaded_model.metrics_names[2], score[2]/10000000))